In [1]:
import numpy as np
import json
import joblib
from umls_search import search_umls
import re
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
def es_recommend(cui,index='disorder',topk=100):
    recom_list = search_umls(cui, index)[:topk]
    # candidates_cui = [i['_source']['cui'] for i in recom_list]
    candidates_str = [
        re.sub(r"-\s",'',
            re.sub(r'[,\'\"]','',
            re.sub(r"\[.*\]","",
            re.sub(r'\s\(.*\)','',i['_source']['str'].lower())
            ))) for i in recom_list
        ]
    # scores = [i['_score'] for i in recom_list]
    return candidates_str

In [5]:
import joblib 
with open('candidates_cui_biosyn_cos_realworld_query_disorder_dictionary.joblib','rb') as f:
    candidates = joblib.load(f)

In [6]:
candidates.keys()

dict_keys(['labels', 'candidates'])

In [7]:
wrong = []
wrong2 = []

In [8]:
for i in range(2824):
    if candidates['labels'][i].sum()==0:
        wrong.append((i, candidates['candidates'][i].tolist()))

In [9]:
import json 
with open('realworld_wrong.json','w') as f:
    f.write(json.dumps(wrong, ensure_ascii=False))

In [10]:
with open('realworld_wrong.json','r') as f:
    wrong = json.load(f)

In [11]:
wrong_index = [i[0] for i in wrong]
candidates = [list(dict.fromkeys(i[1])) for i in wrong]

In [12]:
data = pd.read_csv('../data/query/realworld_query copy.csv')

In [13]:
candidates_strings = []
for candidate in tqdm(candidates,total = len(wrong_index)):
    for c in candidate:
        candidates_strings.extend(es_recommend(c))

100%|██████████| 935/935 [00:16<00:00, 56.29it/s]


In [14]:
en_diffs = []
correct_cuis = []
corrected_index = []
corrected_candidates = []
for i,candidate in tqdm(zip(wrong_index,candidates),total = len(wrong_index)):
    baidu = set(data.iloc[i,]["baidu"].lower().split(' '))
    youdao = set(data.iloc[i,]["youdao"].lower().split(' '))
    tencent = set(data.iloc[i,]["tencent"].lower().split(' '))
    en = set(data.iloc[i,]["en"].lower().split(' '))
    en_diff = min([
        len(baidu.symmetric_difference(en)),
        len(youdao.symmetric_difference(en)),
        len(tencent.symmetric_difference(en)),
    ])
    en_diffs.append(en_diff)
    for c in candidate:
        c_strings = [set(i.split(' ')) for i in es_recommend(c)]
        baidu_diff = [len(baidu.symmetric_difference(c_string)) for c_string in c_strings]
        youdao_diff = [len(youdao.symmetric_difference(c_string)) for c_string in c_strings]
        tencent_diff = [len(tencent.symmetric_difference(c_string)) for c_string in c_strings]    
        all_diff = [10]
        all_diff.extend(baidu_diff)
        all_diff.extend(youdao_diff)
        all_diff.extend(tencent_diff)
        if min(all_diff)<1:
            data.loc[i,"cui"] = c
            corrected_index.append(i)
            corrected_candidates.append(candidate)
            break

100%|██████████| 935/935 [00:16<00:00, 57.97it/s]


In [15]:
for i in corrected_index:
    wrong_index.remove(i)
for i in corrected_candidates:
    candidates.remove(i)

In [16]:
len(wrong_index),len(candidates)

(846, 846)

In [20]:
dictionary = pd.read_csv('../data/dictionary/disorder_dictionary.csv')

In [23]:
dictionary.loc[dictionary['cui']=='C0038045']['terms']

1025439                              sprain
1025440                          sprain nos
1025441                             sprains
1025442    sprain (morphologic abnormality)
1025443                         sprain, nos
Name: terms, dtype: object

In [27]:
candidates_strings = []
for candidate in candidates:
    candidate_strings = []
    for c in candidate:
        candidate_strings.append(list(dictionary.loc[dictionary['cui']==c]['terms']))
    candidates_strings.append(candidate_strings)

In [28]:
real_world_wrong_candidates = [(a,b,c) for a,b,c in zip(wrong_index,candidates,candidates_strings)]

In [29]:
with open('real_world_wrong_candidates.json','w') as f:
    f.write(json.dumps(real_world_wrong_candidates, ensure_ascii=False))

In [63]:
del data['Unnamed: 0.1']

In [64]:
data.to_csv('../data/query/realworld_query_copy.csv',index =True)

In [4]:
import glob
import pandas as pd

In [5]:
files = glob.glob('bert*.csv')
df = []
for file in files:
    df.append(pd.read_csv(file))

In [10]:
df_bert = pd.concat(df,axis=0)

In [ ]:
df_bert.to_csv('../bert_acc.csv',index = False)

In [2]:
import pandas as pd
import numpy as np
import joblib
import argparse
from bert_recom import Bert_Re
from ast import literal_eval
bert_recommend = Bert_Re(use_cuda="True", load_pretrained="False", checkpoint = "coder")

In [3]:
dictionary = pd.read_csv('../data/dictionary/umls_dictionary.csv',dtype = str)

In [4]:
dictionary_token = bert_recommend.tokenize(list(str(i) for i in dictionary["terms"]))

: 

: 